<h1>EEG Analysis for Machine Learning</h1>
<p>
<h2 align="right">using data from Synchronized brainwave dataset</h2>

主題のデータの取得方法，EEGのrawデータから特徴ベクトルに変換する方法

In [1]:
""" データクリーニング """
import json
import pandas as pd

df = pd.read_csv("synchronized-brainwave-dataset/eeg-data.csv")

# 文字列から配列への変換
df.raw_values = df.raw_values.map(json.loads)
df.eeg_power = df.eeg_power.map(json.loads)

### 該当データの取得

In [11]:
relax = df[df.label == 'relax']
math = df[(df.label == 'math1') |
          (df.label == 'math2') |
          (df.label == 'math3') |
          (df.label == 'math4') |
          (df.label == 'math5') |
          (df.label == 'math6') |
          (df.label == 'math7') |
          (df.label == 'math8') |
          (df.label == 'math9') |
          (df.label == 'math10') |
          (df.label == 'math11') |
          (df.label == 'math12') ]
music = df[df.label == 'music']

len(relax)
len(math)

936

In [3]:
math

,Unnamed: 0,id,indra_time,browser_latency,reading_time,attention_esense,meditation_esense,eeg_power,raw_values,signal_quality,createdAt,updatedAt,label
13803,571,1,2015-05-09 23:33:28.977,13403,2015-05-09 16:33:15.726,48,100,"[6650.0, 22871.0, 45854.0, 191253.0, 15457.0, ...","[108.0, 114.0, 120.0, 123.0, 114.0, 104.0, 96....",0,2015-05-09 23:33:02.386,2015-05-09 23:33:02.386+00,math1
13804,23011,4,2015-05-09 23:33:28.985,7691,2015-05-09 19:33:21.374,17,100,"[3131.0, 11610.0, 20863.0, 54613.0, 3711.0, 62...","[88.0, 88.0, 73.0, 74.0, 82.0, 70.0, 58.0, 60....",0,2015-05-09 23:33:13.736,2015-05-09 23:33:13.736+00,math1
13805,12667,2,2015-05-09 23:33:29.017,26,2015-05-09 16:33:29.147,40,48,"[96747.0, 24552.0, 1629.0, 3301.0, 3926.0, 432...","[153.0, 129.0, 118.0, 113.0, 106.0, 108.0, 124...",0,2015-05-09 23:33:29.178,2015-05-09 23:33:29.178+00,math1
13806,6082,14,2015-05-09 23:33:29.052,726,2015-05-09 16:33:28.535,44,69,"[20245.0, 7854.0, 4321.0, 13089.0, 9479.0, 434...","[49.0, 36.0, 28.0, 21.0, 19.0, 9.0, 0.0, 5.0, ...",0,2015-05-09 23:33:27.873,2015-05-09 23:33:27.873+00,math1
13807,3426,12,2015-05-09 23:33:29.109,1504,2015-05-09 16:33:27.868,66,87,"[2740.0, 27325.0, 30234.0, 18751.0, 13404.0, 1...","[11.0, 27.0, 37.0, 17.0, 3.0, 10.0, 25.0, 34.0...",0,2015-05-09 23:33:26.417,2015-05-09 23:33:26.417+00,math1
13808,4894,13,2015-05-09 23:33:29.225,-293,2015-05-09 16:33:29.621,27,53,"[2748564.0, 481537.0, 28312.0, 96849.0, 180628...","[92.0, 107.0, 149.0, 202.0, 228.0, 204.0, 149....",0,2015-05-09 23:33:29.979,2015-05-09 23:33:29.979+00,math1
13809,27666,8,2015-05-09 23:33:29.483,-40,2015-05-09 16:33:29.532,14,70,"[786351.0, 226029.0, 41133.0, 14584.0, 24339.0...","[77.0, 66.0, 64.0, 65.0, 59.0, 59.0, 71.0, 103...",0,2015-05-09 23:33:29.638,2015-05-09 23:33:29.638+00,math1
13810,25824,6,2015-05-09 23:33:29.498,"-1,561",2015-05-09 16:33:31.132,47,30,"[82296.0, 306508.0, 31953.0, 12513.0, 25488.0,...","[-29.0, -42.0, -52.0, -51.0, -39.0, -24.0, -11...",0,2015-05-09 23:33:32.751,2015-05-09 23:33:32.751+00,math1
13811,1262,10,2015-05-09 23:33:29.522,-18,2015-05-09 16:33:29.636,37,70,"[1196729.0, 72232.0, 18068.0, 12065.0, 6169.0,...","[1296.0, 1313.0, 1314.0, 1299.0, 1281.0, 1264....",51,2015-05-09 23:33:29.722,2015-05-09 23:33:29.722+00,math1
13812,2463,11,2015-05-09 23:33:29.599,-179,2015-05-09 16:33:29.976,41,44,"[566294.0, 143887.0, 15984.0, 87967.0, 12868.0...","[-66.0, -43.0, -36.0, -28.0, -4.0, 9.0, 16.0, ...",0,2015-05-09 23:33:30.212,2015-05-09 23:33:30.212+00,math1


## 分類器の定義
### ２値分類

In [4]:
from sklearn.model_selection import cross_val_score
from sklearn import svm
# n交差検定付きSVM
def cross_val_svm (X,y,n):
    clf = svm.SVC()
    scores = cross_val_score(clf, X, y, cv=n)
    return scores  

In [5]:
''' 
ベクトルのリストX,
ラベルのリストy
    を作成する関数
'''
def vectors_labels (list1, list2):
    def label (l):
        return lambda x: l
    X = list1 + list2
    y = list(map(label(0), list1)) + list(map(label(1), list2))
    return X, y

## 特徴ベクトルの作成
512Hzのrawデータの値→FFT→3つのpower spectrum→平均,対数的にbinning→100個の値のベクトルを生成

In [6]:
one_math = math[math['id']==12]
one_relax = relax[relax['id']==12]
X, y = vectors_labels(one_math.eeg_power.tolist(), one_relax.eeg_power.tolist())
cross_val_svm(X,y,7)

array([ 0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5])

In [24]:
""" music versus math """
one_music = music[music['id']==12]
X, y = vectors_labels(one_music.eeg_power.tolist(), one_relax.eeg_power.tolist())
cross_val_svm(X,y,7)

array([ 0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5])

In [7]:
from scipy import stats
from scipy.interpolate import interp1d
import itertools
import numpy as np

def spectrum (vector):
    '''raw EEG dataベクトルのpower spectrum'''
    A = np.fft.fft(vector)
    ps = np.abs(A)**2
    # 正のpower spectrum
    ps = ps[:len(ps)//2]
    return ps

def binned (pspectra, n):
    ''' pspectraの配列をnベクトルに圧縮'''
    l = len(pspectra)
    array = np.zeros([l,n])
    for i,ps in enumerate(pspectra):
        # psの配列の要素数+1
        x = np.arange(1,len(ps)+1)
        # 1D補間法
        f = interp1d(x,ps)#/np.sum(ps))
        array[i] = f(np.arange(1, n+1))
    index = np.argwhere(array[:,0]==-1)
    array = np.delete(array,index,0)
    return array

def feature_vector (readings, bins=100):
    '''
    Create 100, log10-spaced bins for each power spectrum.
    '''
    bins = binned(list(map(spectrum, readings)), bins)
    return np.log10(np.mean(bins, 0))

ex_readings = one_relax.raw_values[:3]
feature_vector(ex_readings)

def grouper(n, iterable, fillvalue=None):
    "grouper(3, 'ABCDEFG', 'x') --> ABC DEF Gxx"
    args = [iter(iterable)] * n
    return itertools.zip_longest(*args, fillvalue=fillvalue)

def vectors (df):
    return [feature_vector(group) for group in list(grouper(3, df.raw_values.tolist()))[:-1]]

In [22]:
X,y = vectors_labels(
    vectors(one_math),
    vectors(one_relax))

cross_val_svm(X,y,7).mean()

0.7857142857142857

In [23]:
""" music versus math """
X,y = vectors_labels(
    vectors(one_music),
    vectors(one_math))

cross_val_svm(X,y,7).mean()

0.8214285714285714

In [21]:
from sklearn import preprocessing
# 標準化
X = preprocessing.scale(X)
cross_val_svm(X,y,7).mean()

0.75

In [10]:
def estimated_accuracy (subject):
    m = math[math['id']==subject]
    r = relax[relax['id']==subject]
    X,y = vectors_labels(vectors(m),vectors(r))
    X=preprocessing.scale(X)
    return cross_val_svm(X,y,7).mean()

[('subject '+str(subj), estimated_accuracy(subj)) for subj in range(1,31)]

[('subject 1', 0.9285714285714286),
 ('subject 2', 0.8214285714285714),
 ('subject 3', 1.0),
 ('subject 4', 0.7857142857142857),
 ('subject 5', 0.9285714285714286),
 ('subject 6', 0.6785714285714286),
 ('subject 7', 0.55952380952380953),
 ('subject 8', 0.8571428571428571),
 ('subject 9', 0.7142857142857143),
 ('subject 10', 0.7142857142857143),
 ('subject 11', 0.8571428571428571),
 ('subject 12', 0.8928571428571429),
 ('subject 13', 0.9642857142857143),
 ('subject 14', 0.9285714285714286),
 ('subject 15', 0.6071428571428571),
 ('subject 16', 0.9285714285714286),
 ('subject 17', 0.66666666666666663),
 ('subject 18', 0.7142857142857143),
 ('subject 19', 0.9642857142857143),
 ('subject 20', 0.8214285714285714),
 ('subject 21', 0.8571428571428571),
 ('subject 22', 0.32142857142857145),
 ('subject 23', 0.7142857142857143),
 ('subject 24', 0.9642857142857143),
 ('subject 25', 0.6071428571428571),
 ('subject 26', 0.52380952380952384),
 ('subject 27', 0.8571428571428571),
 ('subject 28', 0.571

In [26]:
""" music vs. math """
def estimated_accuracy (subject):
    ma = math[math['id']==subject]
    mu = music[music['id']==subject]
    X,y = vectors_labels(vectors(ma),vectors(mu))
    X=preprocessing.scale(X)
    return cross_val_svm(X,y,7).mean()

[('subject '+str(subj), estimated_accuracy(subj)) for subj in range(1,31)]

[('subject 1', 0.8571428571428571),
 ('subject 2', 0.8928571428571429),
 ('subject 3', 1.0),
 ('subject 4', 0.48809523809523808),
 ('subject 5', 0.9642857142857143),
 ('subject 6', 0.7857142857142857),
 ('subject 7', 0.7857142857142857),
 ('subject 8', 0.8214285714285714),
 ('subject 9', 0.7142857142857143),
 ('subject 10', 0.6071428571428571),
 ('subject 11', 0.8571428571428571),
 ('subject 12', 0.75),
 ('subject 13', 0.9642857142857143),
 ('subject 14', 0.9642857142857143),
 ('subject 15', 0.5714285714285714),
 ('subject 16', 0.8928571428571429),
 ('subject 17', 0.6428571428571429),
 ('subject 18', 0.6785714285714286),
 ('subject 19', 0.8928571428571429),
 ('subject 20', 0.8571428571428571),
 ('subject 21', 0.8928571428571429),
 ('subject 22', 0.14285714285714285),
 ('subject 23', 0.9285714285714286),
 ('subject 24', 0.8928571428571429),
 ('subject 25', 0.7857142857142857),
 ('subject 26', 0.75),
 ('subject 27', 0.8214285714285714),
 ('subject 28', 0.7142857142857143),
 ('subject 29'